In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import os

os.chdir("..")

import funciones.parser  as ps

In [33]:
df = ps.branch_data_df('data/prueba_sin_sobrecargas.raw')
# df['BASKV'] = df['I'].str.split().str[-1].str.extract(r'(\d+)').astype(int)
# df['Ybase'] = 100*1000/((df['BASKV']*1000)**2)
df

,I,J,R,X,B
0,'ACO150_B 150.00','PBE150BX 150.00',0.002933,0.009957,0.00376
1,'ACO150_B 150.00','PBE150RX 150.00',0.002933,0.009957,0.00376
2,'ARA150_B 150.00','CLFAUX150_B 150.00',0.015066,0.045404,0.01581
3,'ARA150_B 150.00','TGO150_B 150.00',0.033038,0.099566,0.03467
4,'ARB150P1 150.00','ARB150_B 150.00',0.000920,0.003720,0.00123
...,...,...,...,...,...
138,'SCA150_B 150.00','SCAROC_X 150.00',0.003609,0.013645,0.00517
139,'SGDE.ARG 500.00','SU5500_B 500.00',0.000040,0.000410,0.03890
140,'SJ5500_B 500.00','SU5500_B 500.00',0.001710,0.016060,1.52925
141,'TAB150_B 150.00','TAC150_B 150.00',0.007609,0.022935,0.00797


In [34]:
def get_Admittances(branch_df):
  branch_df['BASKV'] = branch_df['I'].str.split().str[-1].str.extract(r'(\d+)').astype(int)
  branch_df['Ybase'] = 100*1000/((branch_df['BASKV']*1000)**2)
  
  Admitance = np.empty((branch_df.shape[0],2),dtype = 'complex_')
  Admitance[:,0] = (1/(branch_df['R']+branch_df['X']*1j))*branch_df['Ybase']
  Admitance[:,1] = branch_df['B']/2*1j*branch_df['Ybase']
  Admitance = pd.DataFrame(Admitance,columns=['y','ysh'])

  Admitance['I'] = branch_df['I']
  Admitance['J'] = branch_df['J']
  Admitance = Admitance[['I','J','y','ysh']]
  return Admitance

def get_graph(edges_df):
  new_rows = []
  for _, row in edges_df.iterrows():
      source, sink, _, self_loop_weight = row
      new_rows.append({'source': source, 'target': source, 'weight': self_loop_weight})
      new_rows.append({'source': sink, 'target': sink, 'weight': self_loop_weight})
  new_edges_df = edges_df.drop(columns=['ysh'])
  new_edges_df.rename(columns={'I': 'source', 'J': 'target','y': 'weight'}, inplace=True)
  new_rows = pd.DataFrame(new_rows)
  new_edges_df = pd.concat([new_edges_df,new_rows], axis=0)
  new_edges_df_2 = new_edges_df.groupby(['source', 'target'])['weight'].sum().reset_index()
  G = nx.from_pandas_edgelist(new_edges_df_2, edge_attr=True)
  return G

def get_Ybus(G):
  Y = nx.laplacian_matrix(G).toarray()
  for i, node in enumerate(G.nodes()):
      degree = G.degree(node,weight='weight')
      Y[i, i] += (degree - Y[i, i])  / 2
  return Y

In [35]:
Admitance = get_Admittances(df)
G = get_graph(Admitance)
Ybus = get_Ybus(G)
Admitance
# Ybus
# for edge in G.edges():
#     weight_complex = G.edges[edge]['weight']
#     weight_magnitude = abs(weight_complex)
#     G.edges[edge]['weight'] = weight_magnitude

# # Definir el layout y visualizar los nodos
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, node_size=1000, node_color='skyblue', font_size=10, font_color='darkblue', font_weight='bold')

# # Dibujar los enlaces con colores y grosores basados en los valores transformados
# nx.draw_networkx_edges(G, pos, edge_color='gray')

# # Agregar anotaciones o leyendas si es necesario

# # Mostrar la visualización
# plt.show()


,I,J,y,ysh
0,'ACO150_B 150.00','PBE150BX 150.00',0.002655-0.009014j,0.000000e+00+1.833854e-07j
1,'ACO150_B 150.00','PBE150RX 150.00',0.002655-0.009014j,0.000000e+00+1.833854e-07j
2,'ARA150_B 150.00','CLFAUX150_B 150.00',0.000642-0.001935j,0.000000e+00+7.710965e-07j
3,'ARA150_B 150.00','TGO150_B 150.00',0.000293-0.000883j,0.000000e+00+1.690950e-06j
4,'ARB150P1 150.00','ARB150_B 150.00',0.006111-0.024711j,0.000000e+00+5.999043e-08j
...,...,...,...,...
138,'SCA150_B 150.00','SCAROC_X 150.00',0.001767-0.006681j,0.000000e+00+2.521549e-07j
139,'SGDE.ARG 500.00','SU5500_B 500.00',0.026428-0.270887j,0.000000e+00+2.180745e-06j
140,'SJ5500_B 500.00','SU5500_B 500.00',0.000735-0.006903j,0.000000e+00+8.573021e-05j
141,'TAB150_B 150.00','TAC150_B 150.00',0.001271-0.003831j,0.000000e+00+3.887185e-07j


In [32]:
Ybus.shape

(106, 106)

In [27]:
nombres_ordenados = list(G.nodes)
nombres_ordenados = [valor.replace('150.00', '') for valor in nombres_ordenados]
nombres_ordenados = [valor.replace('500.00', '') for valor in nombres_ordenados]
nombres_ordenados = [valor.replace('\'', '') for valor in nombres_ordenados]
nombres_ordenados = [valor.replace(' ', '') for valor in nombres_ordenados]
nombres_ordenados

['ACO150_B',
 'PBE150BX',
 'PBE150RX',
 'ARA150_B',
 'CLFAUX150_B',
 'TGO150_B',
 'ARB150P1',
 'ARB150_B',
 'TYT150_B',
 'ARB150P2',
 'MEB150P1',
 'ART150_B',
 'JPT_150B',
 'RIV150_B',
 'BAY150_B',
 'BON150_B',
 'PAL150_B',
 'TRI150_B',
 'BIF150_B',
 'MVA150_B',
 'PAN150_F',
 'PAZ1501B',
 'PAZ1502B',
 'BOB150_B',
 'YOU150_B',
 'DUR1501F',
 'DUR1502F',
 'PAM150_B',
 'VAL150_B',
 'BRU500_B',
 'MVB500_B',
 'PAL500_B',
 'C.ELIA',
 'SGDE.ARG',
 'SJ5500_B',
 'CGR150_B',
 'MAL150_B',
 'PES150_B',
 'SU5150_B',
 'COE1502P',
 'COL150_B',
 'CON150_B',
 'LIB150_B',
 'MEN150_B',
 'NPA150_B',
 'DOL150_B',
 'MER150_B',
 'PIA150_B',
 'EFI150_X',
 'SVA150_B',
 'EMA150_B',
 'FBE150_B',
 'FBEMER_X',
 'FBESJA_X',
 'SJA150_B',
 'FLO150_B',
 'POL150_B',
 'PRO1501X',
 'PRO1502X',
 'LAD150_B',
 'MEB150_B',
 'MA5500_B',
 'MI5500_B',
 'PAZMAL_X',
 'SCAMAL1X',
 'SCAMAL2X',
 'MDI150_X',
 'STE150_B',
 'TAC150_B',
 'ME5500_B',
 'SC5500_B',
 'MEB150P2',
 'MEL150_B',
 'TAB150_B',
 'ROS150_B',
 'MVB150_B',
 'MVLMVA_X'

In [30]:
datos = 'data/prueba_sin_sobrecargas.raw'
bus_data = ps.bus_data_df(datos)[['NAME', 'BASKV', 'VM', 'VA']]
bus_data = bus_data.set_index('NAME')
bus_data.index = bus_data.index.str.replace('\'', '', regex=False)
bus_data.index = bus_data.index.str.replace(' ', '', regex=False)
bus_data['V (KV)'] = bus_data['BASKV'] * bus_data['VM'] * (np.cos(np.deg2rad(bus_data['VA'])) + np.sin(np.deg2rad(bus_data['VA'])) * 1j)
bus_data = bus_data.reindex(nombres_ordenados)
V = bus_data['V (KV)'].to_numpy()
diagV = np.diag(V)

In [31]:
V

array([154.88386298-15.18540386j, 155.07565742-14.7758506j ,
       154.88859676-15.41215936j, 157.6192992  +3.03415073j,
       156.65569444 +3.87044058j, 159.67759867 +2.42423171j,
       165.98930599-17.87042105j, 165.96775593-17.80422395j,
       166.06714409-20.16956395j, 165.94621049-17.7377562j ,
       165.69030905-16.91026574j, 162.07175462 +4.0158581j ,
       161.98031007 +6.04316139j, 162.33377377-13.3342856j ,
       155.55614228-14.40832682j, 158.50003743-18.90924402j,
       148.55044363 -4.64347145j, 153.94582725-13.62962131j,
       155.42314402-16.47606565j, 156.56950278-13.20062209j,
       156.17826398-14.55475475j, 155.33731868-17.36128292j,
       155.42181814-17.73583309j, 158.48978339-18.9318682j ,
       155.38591289-20.43983796j, 157.29574573-20.88351865j,
       158.35702308-19.08142416j, 161.62924371-19.20731892j,
       164.33924133-20.81740893j, 520.81621538-13.83726115j,
       521.02202966-21.48931897j, 516.75306669 -2.67687574j,
       511.29044689 -2.1